Ingestão de dados


In [37]:
import pandas as pd
import os

#criando dicionário para facilitar o nome dos arquivos 
file_paths = {
    'admissions': 'rawcsvs/ADMISSIONS.csv',
    'microbiology_events': 'rawcsvs/MICROBIOLOGYEVENTS.csv',
    'prescriptions': 'rawcsvs/PRESCRIPTIONS.csv',
    'labevents': 'rawcsvs/LABEVENTS.csv',
    'diagnoses_icd': 'rawcsvs/DIAGNOSES_ICD.csv',
    'patients': 'rawcsvs/PATIENTS.csv',
    'transl-labitems': 'rawcsvs/D_LABITEMS.csv',
    'transl-diag': 'rawcsvs/D_ICD_DIAGNOSES.csv'
}


#carregando arquivos
files = {}
for file_path in file_paths:
    files[file_path] = pd.read_csv(file_paths[file_path])
    
for file_path, file in files.items(): 
    if  not file_path.startswith('transl'):
        file.dropna(inplace=True, subset=['subject_id'])
        file.drop_duplicates(inplace=True)  
        file_name = os.path.basename(file_path)
        file.to_csv(f'cleanedcsv/{file_name}.csv', index=False) 


Criando o dataset para o Modelo


In [38]:
# Lendo os dados dos arrquivos CSV
patients = pd.read_csv(os.path.join('rawcsvs', 'PATIENTS.csv'))
admissions = pd.read_csv(os.path.join('rawcsvs', 'ADMISSIONS.csv'))
microbiology = pd.read_csv(os.path.join('rawcsvs', 'MICROBIOLOGYEVENTS.csv'))

# Mesclando os dados de pacientes e admissões em um novo DataFrame chamado new_data
new_data = (admissions
            .filter(['subject_id', 'admission_type', 'ethnicity', 'diagnosis', 'hospital_expire_flag', 'insurance', 'religion', 'marital_status'])
            .merge(patients, on='subject_id', how='left')
            )

# Mesclando os dados de microbiologia e aplicando one-hot encoding na coluna 'interpretation'
new_data = (
    microbiology
    .filter(['subject_id', 'ab_name', 'interpretation'])
    # One hot encoding the interpretation and overwriting the original 'interpretation' column
    .assign(interpretation=(microbiology['interpretation'] == 'R').astype(int))
    .merge(new_data, on='subject_id', how='left')
)

# Transformando os diagnósticos em formato 'wide' (largura)
aux1 = (
    new_data
    .filter(['subject_id', 'diagnosis'])
    .assign(value=1)
    .pivot_table(index='subject_id', columns='diagnosis', values='value', fill_value=0)
    .reset_index()
)

# Agrupando as colunas restantes por 'subject_id'
aux2 = (
    new_data
    .groupby('subject_id')
    .agg({
        'insurance': 'first',
        'religion': 'first',
        'marital_status': 'first',
        'ethnicity': 'first',
        'gender': 'first',
        'ab_name': 'first',
        'interpretation': 'first'  # Now this is the one-hot encoded interpretation
    })
    .reset_index()
)

# Mesclando aux1 e aux2 para o data set final em um data frame
filtered_data = aux1.merge(aux2, on='subject_id', how='left')

# Exportando os dados filtrados para o CSV
filtered_data.to_csv('improving.csv', index=False)


Engenharia de Features (Feature Engineering)


In [39]:
import re

# Carrega o arquivo CSV 'improving.csv' em um DataFrame chamado filtered_data
filtered_data = pd.read_csv('improving.csv', sep=',')

# Passo 1: Identifica as colunas que não são relacionadas a doenças
non_disease_columns = ['subject_id', 'insurance', 'religion', 'marital_status', 'ethnicity', 'gender', 'ab_name', 'interpretation'] 
# Cria uma lista de todas as colunas que são doenças (excluindo as colunas não relacionadas a doenças)
disease_columns = [col for col in filtered_data.columns if col not in non_disease_columns]

# Passo 2: Mapeia cada doença a seus tipos 
# Define um dicionário que mapeia categorias de doenças (ex: Cardiovascular, Respiratória, etc.) a listas de doenças específicas
disease_type_mapping = {
    'Cardiovascular': ['ACUTE PULMONARY EMBOLISM', 'CONGESTIVE HEART FAILURE', 'CORONARY ARTERY DISEASE', 'STEMI', 'MI CHF', 'BRADYCARDIA', 'VF ARREST', 'PERICARDIAL EFFUSION', 'PULMONARY EDEMA, MI', 'PULMONARY EDEMA\\CATH', 'CRITICAL AORTIC STENOSIS/HYPOTENSION', 'INFERIOR MYOCARDIAL INFARCTION\\CATH'],
    'Respiratory': ['ACUTE RESPIRATORY DISTRESS SYNDROME', 'ASTHMA', 'COPD FLARE', 'PNEUMONIA', 'RESPIRATORY DISTRESS', 'SHORTNESS OF BREATH', 'HYPOXIA', 'TACHYPNEA', 'PLEURAL EFFUSION', 'MEDIASTINAL ADENOPATHY'],
    'Neurological': ['ALTERED MENTAL STATUS', 'SEIZURE', 'STATUS EPILEPTICUS', 'STROKE/TIA', 'INTRACRANIAL HEMORRHAGE', 'HEADACHE', 'FACIAL NUMBNESS', 'BASAL GANGLIN BLEED', 'ALTERED MENTAL STATUS'],
    'Infectious': ['CELLULITIS', 'FEVER', 'SEPSIS', 'URINARY TRACT INFECTION', 'UROSEPSIS', 'PYELONEPHRITIS', 'UTI', 'HIV', 'ABSCESS', 'PNEUMONIA;TELEMETRY', 'SEPSIS; UTI', 'FEVER;URINARY TRACT INFECTION'],
    'Gastrointestinal': ['ABDOMINAL PAIN', 'GASTROINTESTINAL BLEED', 'LOWER GI BLEED', 'UPPER GI BLEED', 'VARICEAL BLEED', 'HEPATITIS', 'LIVER FAILURE', 'CHOLECYSTITIS', 'CHOLANGITIS', 'PANCREATITIS', 'ACUTE CHOLECYSTITIS', 'ACUTE CHOLANGITIS', 'HEPATITIS B', 'ALCOHOLIC HEPATITIS', 'ELEVATED LIVER FUNCTIONS;S/P LIVER TRANSPLANT', 'FAILURE TO THRIVE'],
    'Renal': ['ACUTE RENAL FAILURE', 'CHRONIC KIDNEY DISEASE', 'RENAL CANCER', 'HYPOTENSION, RENAL FAILURE'],
    'Endocrine': ['DIABETES', 'HYPOGLYCEMIA', 'HYPERGLYCEMIA', 'HYPONATREMIA'],
    'Oncology': ['LUNG CANCER', 'BREAST CANCER', 'COLON CANCER', 'LEUKEMIA', 'LYMPHOMA', 'METASTATIC MELANOMA', 'NON SMALL CELL CANCER', 'METASTIC MELANOMA;ANEMIA', 'CHRONIC MYELOGENOUS LEUKEMIA;TRANSFUSION REACTION'],
    'Trauma': ['FRACTURE', 'MOTOR VEHICLE ACCIDENT', 'SUBDURAL HEMATOMA', 'LEFT HIP FRACTURE', 'RIGHT HUMEROUS FRACTURE', 'HUMERAL FRACTURE', 'S/P MOTOR VEHICLE ACCIDENT', 'ACUTE SUBDURAL HEMATOMA'],
    'Other': ['OVERDOSE', 'FAILURE TO THRIVE', 'MEDIASTINAL ADENOPATHY', 'ARREST', 'OVERDOSE', 'AROMEGLEY;BURKITTS LYMPHOMA', 'TRACHEAL ESOPHAGEAL FISTULA', 'TRACHEAL STENOSIS', 'VOLVULUS', 'ANEMIA', 'BURKITTS LYMPHOMA', 'NON SMALL CELL CANCER;HYPOXIA', 'METASTATIC MELANOMA;BRAIN METASTASIS']
}

# Inicializa um dicionário para mapear cada coluna de doença para seu(s) tipo(s) de doença
disease_to_type = {}

# Processa cada coluna de doença
for col in disease_columns:
    # Separa os nomes de doenças caso contenham múltiplas doenças
    diseases = re.split(';|,|/', col)
    diseases = [disease.strip().upper() for disease in diseases]
    types = set()
    for disease in diseases:
         # Encontra o tipo de doença correspondente
        found = False
        for disease_type, disease_list in disease_type_mapping.items():
            if disease in disease_list:
                types.add(disease_type)
                found = True
        if not found:
            types.add('Other')  #  Se a doença não for encontrada, classifica como 'Other'
    disease_to_type[col] = types # Mapeia a coluna da doença para os tipos encontrados


# Passo 3: Para cada tipo de doença, soma as colunas correspondentes de doenças
for disease_type in disease_type_mapping.keys():
    # Obtém todas as colunas que correspondem a esse tipo de doença
    cols = [col for col, types in disease_to_type.items() if disease_type in types]
    if cols:
        # Soma as colunas (como elas são binárias, isso conta o número de doenças desse tipo)
        filtered_data[disease_type] = filtered_data[cols].sum(axis=1)
    else:
        filtered_data[disease_type] = 0  # Se não houver doenças desse tipo, atribui 0

# Remove as colunas originais de doenças do DataFrame
filtered_data.drop(columns=disease_columns, inplace=True)
# Remove a coluna 'subject_id', pois não será mais necessária para as análises a seguir
filtered_data.drop(columns=['subject_id'], inplace=True)    

# Salva o DataFrame filtrado em um novo arquivo CSV
filtered_data.to_csv('improving2.csv', index=False)

# o DataFrame filtered_data contém as colunas não relacionadas a doenças e as colunas somadas por tipo de doença
print(filtered_data.head())


  insurance  religion marital_status               ethnicity gender  \
0  Medicare  CATHOLIC      SEPARATED  BLACK/AFRICAN AMERICAN      F   
1   Private  CATHOLIC         SINGLE   UNKNOWN/NOT SPECIFIED      F   
2  Medicare  CATHOLIC            NaN   UNKNOWN/NOT SPECIFIED      F   
3  Medicare  CATHOLIC       DIVORCED                   WHITE      F   
4  Medicare  CATHOLIC       DIVORCED                   WHITE      M   

        ab_name  interpretation  Cardiovascular  Respiratory  Neurological  \
0    VANCOMYCIN               0             0.0          0.0           0.0   
1           NaN               0             0.0          0.0           0.0   
2           NaN               0             0.0          0.0           0.0   
3           NaN               0             0.0          0.0           0.0   
4  ERYTHROMYCIN               1             0.0          0.0           0.0   

   Infectious  Gastrointestinal  Renal  Endocrine  Oncology  Trauma  Other  
0         1.0              

Treinando e Testando o Modelo


In [40]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE


df = pd.read_csv('improving2.csv')

#  Remove linhas onde o alvo 'interpretation' está ausente (NaN)
df.dropna(subset=['interpretation'], inplace=True)

# Seleciona as colunas de características relevantes e a coluna alvo ('interpretation')
features = ['insurance', 'religion', 'marital_status', 'ethnicity', 'gender', 'ab_name',
            'Cardiovascular', 'Respiratory', 'Neurological', 'Infectious', 'Gastrointestinal', 'Renal',
            'Endocrine', 'Oncology', 'Trauma', 'Other']
target = 'interpretation'

# Codifica as variáveis categóricas e o alvo numérico
label_encoders = {}
for column in features + [target]:
    if df[column].dtype == 'object':
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
        label_encoders[column] = le

# Divide os dados em conjuntos de treino e teste 
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Aplica SMOTE para balancear as classes no conjunto de treino
smote = SMOTE(random_state=42, k_neighbors=1) # SMOTE (Synthetic Minority Over-sampling Technique) é uma técnica que ajuda a lidar com classes desbalanceadas
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Lista de classificadores que serão testados
classifiers = {
    # Random Forest: Conjunto de várias árvores de decisão que trabalham em conjunto. Captura relações complexas nos dados, reduzindo overfitting
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'),

    # Logistic Regression: Modelo linear usado para classificação binária ou multiclasse.
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced'),
    
    # SVM (Support Vector Machine): Classificador que tenta maximizar a margem entre classes. 
    'SVM': SVC(random_state=42, class_weight='balanced'),
    
    # Gradient Boosting: Método de aprendizado em conjunto que combina árvores de decisão sequenciais. Cada árvore tenta corrigir erros das anteriores, melhorando a precisão em dados com padrões complexos.
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    
    # KNN (K-Nearest Neighbors): Classificador que categoriza novos pontos com base nos "K" vizinhos mais próximos
    'KNN': KNeighborsClassifier()
}

# Treina e avalia cada classificador
for name, clf in classifiers.items():
    print(f"\n{name} Results:")
    
    # Treina o classificador no conjunto de treino balanceado (SMOTE)
    clf.fit(X_train_smote, y_train_smote)
    
    # Faz previsões no conjunto de teste
    y_pred = clf.predict(X_test)
    
    # Avalia o classificador com a métrica de acurácia
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred, zero_division=1))



Random Forest Results:
Accuracy: 0.93
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96        27
           1       0.00      0.00      0.00         1

    accuracy                           0.93        28
   macro avg       0.48      0.48      0.48        28
weighted avg       0.93      0.93      0.93        28


Logistic Regression Results:
Accuracy: 0.89
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.89      0.94        27
           1       0.25      1.00      0.40         1

    accuracy                           0.89        28
   macro avg       0.62      0.94      0.67        28
weighted avg       0.97      0.89      0.92        28


SVM Results:
Accuracy: 0.93
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.93      0.96        27
           1       0.33      1.00      0.50         1



Aprimorando modelo


In [41]:
from imblearn.over_sampling import RandomOverSampler

# Como tem poucos 1s e muitos 0s, vamos usar o RandomOverSampler para balancear o dataset
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)


# Train and evaluate each classifier
for name, clf in classifiers.items():
    print(f"\n{name} Results:")
    
    # Train the classifier
    clf.fit(X_train_resampled, y_train_resampled)
    
    # Make predictions
    y_pred = clf.predict(X_test)
    
    # Evaluate the classifier
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred, zero_division=1))



Random Forest Results:
Accuracy: 0.93
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96        27
           1       0.00      0.00      0.00         1

    accuracy                           0.93        28
   macro avg       0.48      0.48      0.48        28
weighted avg       0.93      0.93      0.93        28


Logistic Regression Results:
Accuracy: 0.82
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.85      0.90        27
           1       0.00      0.00      0.00         1

    accuracy                           0.82        28
   macro avg       0.48      0.43      0.45        28
weighted avg       0.92      0.82      0.87        28


SVM Results:
Accuracy: 0.89
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.89      0.94        27
           1       0.25      1.00      0.40         1



Salvando o modelo


In [42]:
# Salvando Pipeline de Dados
import pickle   

# Salvando modelo em um arquivo model.pkl
with open('model.pkl', 'wb') as model_file:
    pickle.dump((classifiers, label_encoders), model_file)   
    


Páginas HTML


In [43]:
from flask import Flask, render_template, request, jsonify, redirect, url_for, session
import json
import pickle
import secrets
import plotly.express as px
import plotly.io as pio

# Cria uma aplicação Flask e define uma chave secreta para a sessão
app = Flask(__name__)
app.secret_key = secrets.token_urlsafe(16)

# Função usada para encontrar o antibiótico mais próximo do nome pesquisada
def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)

    # Inicializa a matriz de distância
    previous_row = range(len(s2) + 1)
    
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

# Função para contar resistências de antibióticos por doença com base no medicamento mais próximo
def dis_res_count(closest_drug):
    # Filtra as doenças baseadas no medicamento mais próximo
    diseases = files['microbiology_events'][files['microbiology_events']['ab_name'] == closest_drug]['org_name'].unique()
    
    # Dicionário para armazenar a contagem de resistência por doença
    dis_res_count = {}

    # Itera sobre cada doença e conta as interpretações 'R', 'I', 'S'
    for disease in diseases:
        disease_res = files['microbiology_events'][
            (files['microbiology_events']['ab_name'] == closest_drug) & 
            (files['microbiology_events']['org_name'] == disease)
        ]['interpretation']
        
        # Inicializa a contagem de 'R', 'I', 'S'
        counts = {'R': 0, 'I': 0, 'S': 0}
        
        # Atualiza as contagens com base nas interpretações
        counts.update(disease_res.value_counts().to_dict())
        
        # Armazena a contagem da doença atual
        dis_res_count[disease] = counts

    return dis_res_count


# Função para gerar um gráfico de resistência usando Plotly
def plot_resistance_graph(resistencias, closest_drug):
    # Prepara dados para o gráfico em formato sunburst
    data = {
        'character': [],
        'parent': [],
        'value': [],
        'ids': [],
        'labels': [],
    }

    # Adiciona o antibiótico como raiz do gráfico
    data['character'].append(closest_drug)
    data['ids'].append(closest_drug)
    data['labels'].append(closest_drug)
    data['parent'].append('')
    data['value'].append(sum([sum(counts.values())
                         for counts in resistencias.values()]))

    # Preenche dados para cada bactéria e contagens de resistência
    for bacteria, counts in resistencias.items():
        # Adiciona bactérias a partir dos antibióticos
        data['ids'].append(bacteria)
        data['labels'].append(bacteria)
        data['character'].append(bacteria)
        data['parent'].append(closest_drug)
        data['value'].append(sum(counts.values()))

        # Adiciona níveis de resistência a partir de cada bactéria
        for i, (interpretation, count) in enumerate(counts.items()):
            if count > 0:
                unique_interpretation = f"{interpretation}_{bacteria}"
                data['ids'].append(unique_interpretation)
                data['labels'].append(interpretation)
                data['character'].append(unique_interpretation)
                data['parent'].append(bacteria)
                data['value'].append(count)

   # Cria o gráfico sunburst
    fig = px.sunburst(
        data,
        ids='ids',          # Unique IDs
        names='labels',      # Display names
        parents='parent',   # Parent-child relationships using IDs
        values='value',     # Values for each level
        color='labels',      # Color based on label names
        color_discrete_map={'R': 'red', 'I': 'yellow', 'S': 'green'},
        height=800,
        width=800
    )

    # Atualiza o layout para transparência e tamanho
    fig.update_layout(
        template=None,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
    )

    # Converte o gráfico para HTML
    graph_html = pio.to_html(fig, full_html=False)
    return graph_html


# Rota principal (página inicial) com busca de antibiótico
@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'GET':
        return render_template('busca.html')
    elif request.method == 'POST':
        ab_name = request.form.get('ab_name') # Recebe o nome do antibiótico do formulário

        def search(term):
            all_drugs = files['microbiology_events']['ab_name'].dropna().unique()

            closest_drug = None    
            for drug in all_drugs:
                dist = levenshtein_distance(term, drug)
                if closest_drug is None or dist < closest_drug[1]:
                   closest_drug = (drug, dist)
            return closest_drug
        

        if ab_name:
            droga = search(ab_name)
            session['droga'] = droga[0]  # Armazena apenas o nome da droga na sessão
            return redirect(url_for('resposta'))
        else:
            return jsonify({"error": "Nome do antibiótico não informado"}), 400

# Rota para a página secundaria (resposta)

@app.route('/resposta')
def resposta():
    closest_drug = session.get('droga')  # Recupera a droga da sessão

    # Verifica se o medicamento foi encontrado
    if closest_drug:
        # Chama a função que calcula as contagens de resistência
        resistencias = dis_res_count(closest_drug)
        graph_html = plot_resistance_graph(resistencias, closest_drug) # Rota para a página de resposta que exibe a resistência
        print(resistencias)
    else:
        resistencias = {}
        graph_html = None

    # Renderiza o template com os dados de resistência
    return render_template('resposta.html', closest_drug=closest_drug, resistencias=resistencias, graph_html=graph_html)


# Função para predição de resistência via AJAX
@app.route('/predict', methods=['POST'])
def run_prediction():
    if request.method == "POST":
        byte_string = request.data.decode('utf-8')
        data_dict = {item['name']: item['value'] for item in json.loads(byte_string)}
        
        # Transforma o dicionário em DataFrame para processar
        data = pd.DataFrame(data_dict, index=[0])
        selected_columns = ["Cardiovascular", "Respiratory", "Neurological", "Infectious", 
                        "Gastrointestinal", "Renal", "Endocrine", "Oncology", "Trauma", "Other"]
        
        data[selected_columns] = data[selected_columns].replace({'Low': 0, 'Medium': 1, 'High': 2})

        # Define valores únicos para colunas categóricas
        all_ins = files['admissions']['insurance'].dropna().unique()
        all_rels = files['admissions']['religion'].dropna().unique()
        all_mar = files['admissions']['marital_status'].dropna().unique()
        all_eth = files['admissions']['ethnicity'].dropna().unique()
        all_gend = files['patients']['gender'].dropna().unique()
        all_drugs = files['microbiology_events']['ab_name'].dropna().unique()


        col_to_all = {
            'insurance': all_ins,
            'religion': all_rels,
            'marital_status': all_mar,
            'ethnicity': all_eth,
            'gender': all_gend,
            'ab_name': all_drugs
        }

        # Define as colunas categóricas do conjunto de dados que serão comparadas para encontrar correspondências próximas
        text_columns = ['insurance', 'religion', 'marital_status', 'ethnicity', 'gender', 'ab_name']


        # Dicionário para armazenar as correspondências mais próximas para cada coluna
        closest_received_data = {}

        # Itera sobre o conjunto de dados recebido, encontrando a menor distância entre cada valor recebido e os valores do conjunto de dados de treino
        for text_col in text_columns:
            closest_match = None
            closest_distance = float('inf')  # Inicia com um valor alto para comparação inicial
            for received_col in data[text_col]:  # Itera sobre os valores recebidos para a coluna atual
                for all_col in col_to_all[text_col]:  # Itera sobre os valores do conjunto de dados treinado para a coluna atual
                    dist = levenshtein_distance(received_col, all_col)

                    # Atualiza o valor mais próximo se uma correspondência mais próxima for encontrada
                    if dist < closest_distance:
                        closest_match = all_col
                        closest_distance = dist

            # Armazena a correspondência mais próxima para a coluna atual
            closest_received_data[text_col] = closest_match
        # Converte as correspondências mais próximas para um DataFrame
        text_df = pd.DataFrame(closest_received_data, index=[0])
        # Concat as colunas categóricas ajustadas com os dados restantes selecionados para a previsão
        data_to_predict = pd.concat([data[selected_columns], text_df], axis=1).iloc[0:1]
        # Define a ordem das colunas no conjunto de dados de previsão, alinhando-as com o modelo de machine learning
        data_to_predict = data_to_predict[[
            'insurance', 'religion', 'marital_status', 'ethnicity', 'gender', 'ab_name',
            'Cardiovascular', 'Respiratory', 'Neurological', 'Infectious', 
            'Gastrointestinal', 'Renal', 'Endocrine', 'Oncology', 'Trauma', 'Other'
        ]].iloc[0:1]
    

        # Carrega os modelos e os codificadores de rótulos
        models, label_encoders = pickle.load(open('model.pkl', 'rb'))

        # Transforma os valores categóricos para o formato esperado pelo modelo, utilizando os codificadores de rótulos
        for column, encoder in label_encoders.items():
            data_to_predict[column] = encoder.transform(data_to_predict[column])

        # Inicializa a lista de previsões
        predictions = []
        for name, classifier in models.items():
            prediction = classifier.predict(data_to_predict)
            predictions.append(prediction)

        # Calcula a pontuação percentual de previsões positivas (resistência encontrada)
        score = sum(pred == 1 for pred in predictions)
        percent_score = (score / len(predictions)) * 100

        # Retorna a pontuação percentual em formato JSON
        return jsonify({'score': percent_score[0]}), 200


if __name__ == '__main__':
    app.run() 






 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Oct/2024 22:05:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:05:03] "GET /static/estilos.css HTTP/1.1" 304 -
127.0.0.1 - - [26/Oct/2024 22:05:03] "GET /static/busca.js HTTP/1.1" 304 -
127.0.0.1 - - [26/Oct/2024 22:05:05] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [26/Oct/2024 22:05:05] "GET /resposta HTTP/1.1" 200 -


{'ENTEROBACTERIACEAE': {'R': 0, 'I': 0, 'S': 2}, 'ESCHERICHIA COLI': {'R': 0, 'I': 0, 'S': 6}, 'SERRATIA MARCESCENS': {'R': 0, 'I': 0, 'S': 1}, 'KLEBSIELLA PNEUMONIAE': {'R': 0, 'I': 0, 'S': 1}, 'KLEBSIELLA OXYTOCA': {'R': 0, 'I': 0, 'S': 1}, 'ACINETOBACTER BAUMANNII COMPLEX': {'R': 1, 'I': 2, 'S': 4}}


127.0.0.1 - - [26/Oct/2024 22:05:05] "GET /resposta HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:05:05] "GET /static/resposta.css HTTP/1.1" 304 -


{'ENTEROBACTERIACEAE': {'R': 0, 'I': 0, 'S': 2}, 'ESCHERICHIA COLI': {'R': 0, 'I': 0, 'S': 6}, 'SERRATIA MARCESCENS': {'R': 0, 'I': 0, 'S': 1}, 'KLEBSIELLA PNEUMONIAE': {'R': 0, 'I': 0, 'S': 1}, 'KLEBSIELLA OXYTOCA': {'R': 0, 'I': 0, 'S': 1}, 'ACINETOBACTER BAUMANNII COMPLEX': {'R': 1, 'I': 2, 'S': 4}}


127.0.0.1 - - [26/Oct/2024 22:05:06] "GET /static/resposta.js HTTP/1.1" 304 -
C:\Users\bitan\AppData\Local\Temp\ipykernel_23952\3612384985.py:179: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

127.0.0.1 - - [26/Oct/2024 22:05:14] "POST /predict HTTP/1.1" 200 -
